# Práctica 1.A Toma de contacto con AIMA

#### INTEGRANTES DEL GRUPO
PABLO MARTINEZ DOMINGO

CESAR CARLOS RUBIO PASTOR

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se **representan** algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  

Cuando termines los ejercicios entrega este archivo en el campus. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [1]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *

In [2]:
#Cargamos el módulo de math que usaremos para calcular heurística sqrt_manhattan
import math

In [3]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Dado un estado, devuelve las posibles acciones de ese estado ?¿?¿?¿? no se si es asi 100%"""
        """donde vamos a tener las acciones"""
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """dado una accion y un estado ejecuta l accion y devuelve el estado siguiente"""
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [4]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [5]:
p =Jarras()
p.initial

(0, 0)

In [6]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [7]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [8]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [9]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [10]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [11]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [12]:
misioneros.initial

(3, 3, 0)

In [13]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [14]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self, estado):
        pos_hueco = estado.index(0) # busco la posicion del 0
        accs=list()
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        
        return accs     

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        
        ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco - 3]
            l[pos_hueco - 3] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    
    #redefinimos la funcion coste_de_aplicar_accion para que muestre por pantalla "No se puede realizar esta acción"
    #en caso de producirse una acción que no se puede realizar y 1 si se puede realizar la acción
    def coste_de_aplicar_accion(self, estado, accion):
        if accion in self.actions(estado):
            return 1
        else:
            return "No se puede realizar esta acción."

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [15]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [16]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

La siguiente llamada (mover hueco abajo) es una llamada forzada por eso da fallo (se le llama con un estado inicial que no cumple la precondiccion para esa accion). En este caso seria q el 0 esta en las celdas de abajo. El algoritmo de busqueda tiene en cuenta las posibles acciones pera cada estado y no se producen estos errores.

In [17]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

In [18]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [19]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

'No se puede realizar esta acción.'

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

estos son los algoritmos de busqueda, como se utilizan con los ejercicios que hemos programados
si un algoritmo es completo, termin y siempre encuentra solucion
si tarda mucho puede que el algoritmo sea erroneo y no termina

cargamos el fichero aima-python en el directorio en el que esta esta practica para importar search, que lo vamos a utilizar para buscar dentroo de nuestros ejercicios. algoritmos de busqueda -> search

In [20]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [21]:
## resolvemos el problema de las jarras con el método de búsqueda en anchura.  

In [22]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [23]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8

In [24]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura breadth_first_tree_search(misioneros).solution()

In [25]:
#Este estado no es resoluble
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [26]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

#### ALGORITMOS DE BÚSQUEDA EN ANCHURA

Busqueda en profundidad sin control de repeticiones

In [ ]:
#Búsqueda en anchura sin control de repeticiones
breadth_first_tree_search(p8).solution()
# La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. ¿no debería terminar? escribe al final del ejercicio tus conclusiones.

#Este estado inicial del puzzle no tiene solución porque tiene un número impar de inversiones (11 inversiones).
#De esta manera, al no tener control de repetidos, nunca termina aunque sea una busqueda completa.

#El estado inicial es uno que no consta de solucion, por lo que al no contar con las repeticiones(lista de explored)
#queda atrapado en un loop

In [27]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()
#En este caso el algoritmo de búsqueda encuentra la solución porque el estado inicial del puzzle tiene solución.

['Mover hueco derecha', 'Mover hueco derecha']

In [28]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [29]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

Búsqueda en anchura con control de repetidos

In [30]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

#### ALGORITMOS DE BÚSQUEDA EN PROFUNDIDAD

Búsqueda en profundidad con control de repeticiones

In [193]:
depth_first_graph_search(estado).solution()
#Tarda entornoa 20 mins e encontrar la solución

KeyboardInterrupt: 

Búsqueda en profundidad sin control de repeticiones

In [194]:
#Con control de repetidos tarda mucho, por lo que sin control de repetidos tarda mucho más. Por eso no es necesario
#ejecutar esta busqueda
#depth_first_tree_search(estado).solution()

#### Conclusiones de los algorítmos de búsqueda:

Distinguimos entre los tres estados probados:
    Para el estado (2, 8, 3, 1, 6, 4, 7, 0, 5), no hay solucion puesto que hemos visto que el numero de inversiones es impar (11 inversiones).
    Para el estado (1, 2, 3, 4, 5, 6, 0, 7, 8), la búsqueda ciega tanto en profundidad como en anchura encuentran solución relativamente pronto, dado que es un estado inicial con pocas inversiones.
    Para el estado (2, 4, 3, 1, 5, 6, 7, 8, 0), la búsqueda en anchura consigue solución rapidamente, a diferencia de la búsqueda en profundidad, que tarda mucho más en comparación.

### Ejercicio 3:  Definición de heurísticas

#### Para el problema de los misioneros define una heurística y comenta sus propiedades
#### Para el problema del puzle de 8 se pide definir al menos las siguientes funciones heurísticas:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node): maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [30]:
# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    
    ln = 0
    for i in range(0,9):
            if node.state[i] != 0:
                if node.state[i] != goal[i]: #Si no es el 0 (hueco)
                    ln += 1
                    
    return ln


def manhattan(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    
    mhd = 0
    for i in range(0,9):
        if node.state[i] != 0:
            j = goal.index(state[i])
            if i != j:
                filaEstado = i / 3 
                columnaEstado = i % 3
                filaFinal = j / 3
                columnaFinal = j % 3
                
                mhd += abs(filaEstado - filaFinal) + abs(columnaEstado - columnaFinal)
    
    return mhd

def sqrt_manhattan(node):
    ##state = node.state
    
    return math.sqrt(manhattan(node))

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    
    return max(score1, score2)
        

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente **estado inicial** y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [32]:
puzle = Ocho_Puzzle((2,4,3,1,5,6,7,8,0))

In [33]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [34]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

#### Búsqueda de coste uniforme

In [35]:
uniform_cost_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [36]:
%%timeit
uniform_cost_search(puzle)
#con busqueda en coste uniforme tarda de media 15.9 ms

15.9 ms ± 688 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Búsqueda primero el mejor con heurística linear

In [37]:
best_first_graph_search(puzle, linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [38]:
%%timeit
best_first_graph_search(puzle, linear)
#con búsqueda el primero mejor con heurística linear tarda de media 109 ms

109 ms ± 6.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Búsqueda primero el mejor con heurística manhattan

In [39]:
best_first_graph_search(puzle, manhattan).solution()

['Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [40]:
%%timeit
best_first_graph_search(puzle, manhattan)
# con busqueda primero el mejor con heurística manhattan tarda de media 0.335 ms ± 30.5 µs

335 µs ± 30.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Búsqueda primero el mejor con heurística sqrt_manhattan

In [41]:
best_first_graph_search(puzle, sqrt_manhattan).solution()

['Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [42]:
%%timeit
best_first_graph_search(puzle, sqrt_manhattan)
#con busqueda primero el mejor con heurística sqrt_manhattan tarda de media 0.335 ms ± 27.6 µs

335 µs ± 27.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Búsqueda primero el mejor con heurística max_heuristic

In [43]:
best_first_graph_search(puzle, max_heuristic).solution()

['Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha']

In [44]:
%%timeit
best_first_graph_search(puzle, max_heuristic)
#con busqueda primero el mejor con heurística max_heuristic tarda de media 0.423 ms

423 µs ± 34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Búsqueda A* sin heurística

In [45]:
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [46]:
%%timeit
astar_search(puzle)
#con busqueda A* sin heurística tarda de media 15.2 ms

15.2 ms ± 437 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Búsqueda A* con heurística linear

In [47]:
astar_search(puzle, linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [48]:
%%timeit
astar_search(puzle, linear)
#con búsqueda A* con heurística linear tarda de media 0.411 ms

411 µs ± 30.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Búsqueda A* con heurística manhattan

In [49]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [50]:
%%timeit
astar_search(puzle,manhattan)
#con búsqueda A* con heurístic manhattan tarda de media 0.456 ms

456 µs ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Búsqueda A* con heurística sqrt_manhattan

In [51]:
astar_search(puzle, sqrt_manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [52]:
%%timeit
astar_search(puzle, sqrt_manhattan)
#con búsqueda A* con heurística sqrt_manhattan tarda de media 2.45 ms

2.45 ms ± 95.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Búsqueda A* con heurística max_heuristic

In [53]:
astar_search(puzle, max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [54]:
%%timeit
astar_search(puzle, max_heuristic)
#con búsqueda A* con heurística max_heuristic tarda de media 0.538 ms

538 µs ± 35.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Tabla de resultados

|                         Método | Tiempo (ms) |
|:-------------------------------|:------------|
|                 Coste Uniforme |       15.9  |
|         Primero Mejor (linear) |       109   |
|      Primero Mejor (manhattan) |       0.335 |
| Primero Mejor (sqrt_manhattan) |       0.335 |
|  Primero Mejor (max_heuristic) |       0.423 |
|                             A* |       15.2  |
|                    A* (linear) |       0.411 |
|                 A* (manhattan) |       0.456 |
|            A* (sqrt_manhattan) |       2.45  |
|             A* (max_heuristic) |       0.538 |

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo.Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [55]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [56]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

4.62 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

2.65 ms ± 142 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

48.6 ms ± 2.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

2.92 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

# ----------------------------------------------------------------------------

La mejor heurística es la Manhattan, seguida de la Lineal.

Respecto a la heurística Máximo, que coge el máximo entre la heurística Lineal y la Manhattan, esta da un resultado parecido a la Manhattan, dado que la Lineal cuenta el número de piezas descoladas, si este es N, entonces la suma de las distancias Manhattan de las piezas a sus posiciones será por lo menos N.

Por último, la heurística sqrt_manhattan es la peor heurística de todas con diferencia.

# ----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [28]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

In [61]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8 = Ocho_Puzzle(estado_inicial)
p8p = Problema_con_Analizados(p8)

In [62]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [63]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [66]:
p8p.check_solvability(p8p.initial)

True

Utilizamos la heurística Manhattan que es la más eficiente (ejercicio anterior)

In [67]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [68]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [69]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [70]:
p8p.analizados

2

In [24]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p = Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol = algoritmo(p8p,h).solution()
        else: 
            sol = algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [72]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


### Ejercicio 5:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 1 | 3 |   | 1 | 0 | 3 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 4 | 8 | 6 |   | 4 | 8 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 2 | 5 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  
 
 

-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
    Anchura                            L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA= 
   
    Profundidad                        L=           L=           L=          L=  
                                       T=           T=           T=          T= 
                                       NA=          NA=          NA=         NA= 
                                       
    Coste Uniforme                     L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA= 
                                       
    Primero el mejor (linear)          L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                            
    Primero el mejor (manhattan)       L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (linear)                        L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (manhattan)                    L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=

 -----------------------------------------------------------------------------------------




In [26]:
E1 = (2,1,3,4,8,6,7,0,5)
E2 = (1,0,3,4,8,6,7,2,5)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)

En vez de utilizar timeit que calcula el tiempo medio de entre muchas ejecucciones, hemos hecho uso de la libreria time para calcular lo que tarda una ejecucción. Hemos optado por esta idea dado que hay algunas busquedas que tardan mucho y no nos conviene hacer varias ejecucciones para hayar el tiempo medio.

#### Búsqueda en anchura

In [77]:
import time
start= time.time()

resuelve_ocho_puzzle((E1),breadth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 17. Nodos analizados: 13634
9.62525987625122


In [78]:
import time
start= time.time()

resuelve_ocho_puzzle((E2),breadth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 1255
0.13361144065856934


In [79]:
import time
start= time.time()

resuelve_ocho_puzzle((E3),breadth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 20. Nodos analizados: 57057
176.9430000782013


In [80]:
import time
start= time.time()

resuelve_ocho_puzzle((E4),breadth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 3. Nodos analizados: 13
0.0010187625885009766


#### Búsqueda en profundidad

In [81]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, depth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco der

In [82]:
#Tarda muchísimo, no se han conseguido datos de tipo de búsqueda con este estado
import time
start= time.time()

resuelve_ocho_puzzle(E2, depth_first_graph_search)
print(time.time()-start)

KeyboardInterrupt: 

In [83]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, depth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquie

In [84]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, depth_first_graph_search)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: depth_first_graph_search
Longitud de la solución: 27. Nodos analizados: 28
0.0013570785522460938


#### Búsqueda de coste uniforme

In [85]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, uniform_cost_search)
print(time.time()-start)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 17. Nodos analizados: 14092
91.63569188117981


In [86]:
import time
start= time.time()

resuelve_ocho_puzzle(E2, uniform_cost_search)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 11. Nodos analizados: 870
0.3307950496673584


In [87]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, uniform_cost_search)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 20. Nodos analizados: 48428
1101.0888712406158


In [88]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, uniform_cost_search)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: uniform_cost_search
Longitud de la solución: 3. Nodos analizados: 13
0.0015270709991455078


#### Búsqueda primero mejor con heurística linear

In [89]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, best_first_graph_search, linear)
print(time.time()-start)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 39. Nodos analizados: 171

In [90]:
import time
start= time.time()

resuelve_ocho_puzzle(E2, best_first_graph_search, linear)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 33. Nodos analizados: 78
0.004554033279418945


In [91]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, best_first_graph_search, linear)
print(time.time()-start)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hue

In [92]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, best_first_graph_search, linear)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
0.0005068778991699219


#### Búsqueda primero mejor con heurística manhattan

In [93]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, best_first_graph_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco a

In [94]:
import time
start= time.time()

resuelve_ocho_puzzle(E2, best_first_graph_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 37. Nodos analizados: 64
0.0035386085510253906


In [95]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, best_first_graph_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco der

In [96]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, best_first_graph_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
0.0005121231079101562


#### Búsqueda A* con eurística linear

In [97]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, astar_search, linear)
print(time.time()-start)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 17. Nodos analizados: 835
0.29422593116760254


In [98]:
import time
start= time.time()

resuelve_ocho_puzzle(E2, astar_search, linear)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 11. Nodos analizados: 74
0.01116800308227539


In [99]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, astar_search, linear)
print(time.time()-start)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 20. Nodos analizados: 3230
4.039474010467529


In [100]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, astar_search, linear)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 3. Nodos analizados: 4
0.0010311603546142578


#### Búsqueda A* con eurística manhattan

In [101]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, astar_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 17. Nodos analizados: 278
0.05699729919433594


In [102]:
import time
start= time.time()

resuelve_ocho_puzzle(E2, astar_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 11. Nodos analizados: 28
0.0020389556884765625


In [103]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, astar_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 20. Nodos analizados: 931
0.36600804328918457


In [104]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, astar_search, manhattan)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 3. Nodos analizados: 4
0.0005147457122802734


TABLA ACTUALIZADA(el tiempo esta expresado en segundos)

-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
                                       
    Anchura                            L= 17        L= 11        L= 20       L= 3
                                       T= 9.63      T= 0.134     T= 176.94   T= 0.001 
                                       NA= 13634    NA= 1255     NA= 57057   NA= 13
   
    Profundidad                        L= 7977      L= NAN       L= 39720    L= 27 
                                       T= 15.19     T= NAN       T= 385.91   T= 0.00135
                                       NA= 8198     NA= NAN      NA= 42926   NA= 28
                                       
    Coste Uniforme                     L= 17        L= 11          L= 20     L= 3
                                       T= 91.6356   T= 0.331     T= 1101.09  T= 0.00152  
                                       NA= 14092    NA= 870      NA= 48428   NA= 13
                                       
    Primero el mejor (linear)          L= 39        L= 33        L= 76       L= 3
                                       T= 0.0189    T= 0.0046    T= 0.2398   T= 0.00051 
                                       NA= 171      NA= 78       NA= 747     NA= 4
                                                                            
    Primero el mejor (manhattan)       L= 47        L= 64        L= 227      L= 3
                                       T= 0.0296    T= 0.00354   T= 0.0463   T= 0.00051 
                                       NA= 214      NA= 37       NA= 82      NA= 4
                                                                             
    A* (linear)                        L= 17        L= 11        L= 20       L= 3
                                       T= 0.294     T= 0.0112    T= 4.0395   T= 0.00103 
                                       NA= 835      NA= 74       NA= 3230    NA= 4
                                                                             
    A* (manhattan)                     L= 17        L= 11        L= 20       L= 3 
                                       T= 0.0569    T= 0.00203   T= 0.366    T= 0.00051
                                       NA= 278      NA= 28       NA= 931     NA= 4

 -----------------------------------------------------------------------------------------


### Ejercicio 6:  En el ejercicio anterior hemos hecho pruebas con distintos estados iniciales. Explica cómo afecta a la resolución del problema si lo que cambiamos es el estado objetivo.
####  En vez de  goal=(1, 2, 3, 4, 5, 6, 7, 8, 0) queremos usar goal2=(1, 2, 3, 4, 0, 5, 6, 7, 8)
####  No es necesario repetir los experimentos pero sí justificar convenientemente la respuesta dada.



###  -----------------------------------------------------------------------------------------------------------------



###  -----------------------------------------------------------------------------------------------------------------


### Ejercicio 7: (opcional) Definir nuevas heurísticas (más informadas) y completa una nueva fila de la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [ ]:
# Proponemmos como ejemplo la heurística h3 que se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.

# Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.

         

# ---------------------------------------------------------------------------



#### Heurística sequential

Tenemos que tener en cuenta el orden propuesto (agujas del reloj) y aumentamos un contador en 2 ó 1 dependiendo de la posición del elemento adyacente al que estamos evaluando, de tal manera que:
 Si para cualquier elemento, su adyacente no es el correcto, aumentamos el contador en 2.
 Si para cualquier elemento, este ocupa la posición donde tiene que estar el hueco, aumentamos el contador en 1.

El estado objetivo del puzzle:

              +---+---+---+
              | 1 | 2 | 3 |
              +---+---+---+
              | 4 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+

Definimos un orden secuencial en el tablero (en función de las agujas del reloj):

              +---+---+---+
              | 4 | 5 | 6 |
              +---+---+---+
              | 3 | 0 | 7 | 
              +---+---+---+
              | 2 | 1 | 8 |
              +---+---+---+
              
Tomando las posiciones del vector que representa la tabla:

              +---+---+---+
              | 0 | 1 | 2 |
              +---+---+---+
              | 3 | 4 | 5 |
              +---+---+---+
              | 6 | 7 | 8 |
              +---+---+---+
              

Con estos dos preceptos, podemos identificar las posiciones adyacentes de las posiciones del vector:

| Posición | Adyacente |
|:--------:|:---------:|
|        0 |         1 |
|        1 |         2 |
|        2 |         5 |
|        3 |         0 |
|        4 |         7 |
|        5 |         8 |
|        6 |         3 |
|        7 |         6 |
|        8 |         - |


In [20]:
def nextPosition(pos):
    switcher = {
        0: 1,
        1: 2,
        2: 5,
        3: 0,
        4: 7,
        5: 8,
        6: 3,
        7: 6,
    }
    
    return switcher.get(pos, -1)

def nextPiece(piece):
    switcher = {
        1: 2,
        2: 3,
        3: 6,
        4: 1,
        5: 8,
        6: 0,
        7: 4,
        8: 7
    }
    
    return switcher.get(piece, -1)

def sequential(node, l=1):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    
    score = 0
    for piece in range(1,9):
        
        pos = state.index(piece)
        
        if pos == 8:
            score += 1*l
        else:
            nextPos = nextPosition(pos) ## next sequential position
            adjPiece = state[nextPos]
            
            if nextPiece(piece) != adjPiece:
                score += 2*l
                
    return score

Por ejemplo, para el estado inicial dado por:

          +---+---+---+
          | 2 | 1 | 3 |
          +---+---+---+
          | 4 | 5 | 6 |
          +---+---+---+
          | 7 | 8 | 0 |
          +---+---+---+

su puntuación es 6:

    El adyacente al 1 es el 3 (debería ser el 2) => +2
    El adyacente al 2 es el 1 (debería ser el 3) => +2
    El adyacente al 4 es el 2 (debería ser el 1) => +2



In [21]:
sequential(Node((2,1,3,4,5,6,7,8,0)))

6

Definimos la función de heurística pedida en el enunciado (h3):

In [22]:
def h3(node):

    return manhattan(node) + sequential(node)

Probamos la nueva heurística con los estados del ejercicio anterior

In [31]:
import time
start= time.time()

resuelve_ocho_puzzle(E1, astar_search, h3)
print(time.time()-start)

Solución: ['Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 19. Nodos analizados: 87
0.013788461685180664


In [32]:
import time
start= time.time()

resuelve_ocho_puzzle(E2, astar_search, h3)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 11. Nodos analizados: 43
0.006656169891357422


In [33]:
import time
start= time.time()

resuelve_ocho_puzzle(E3, astar_search, h3)
print(time.time()-start)

Solución: ['Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 20. Nodos analizados: 324
0.08470559120178223


In [34]:
import time
start= time.time()

resuelve_ocho_puzzle(E4, astar_search, h3)
print(time.time()-start)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: h3
Longitud de la solución: 3. Nodos analizados: 4
0.000514984130859375


-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
                                       
    Anchura                            L= 17        L= 11        L= 20       L= 3
                                       T= 9.63      T= 0.134     T= 176.94   T= 0.001 
                                       NA= 13634    NA= 1255     NA= 57057   NA= 13
   
    Profundidad                        L= 7977      L= NAN       L= 39720    L= 27 
                                       T= 15.19     T= NAN       T= 385.91   T= 0.00135
                                       NA= 8198     NA= NAN      NA= 42926   NA= 28
                                       
    Coste Uniforme                     L= 17        L= 11          L= 20     L= 3
                                       T= 91.6356   T= 0.331     T= 1101.09  T= 0.00152  
                                       NA= 14092    NA= 870      NA= 48428   NA= 13
                                       
    Primero el mejor (linear)          L= 39        L= 33        L= 76       L= 3
                                       T= 0.0189    T= 0.0046    T= 0.2398   T= 0.00051 
                                       NA= 171      NA= 78       NA= 747     NA= 4
                                                                            
    Primero el mejor (manhattan)       L= 47        L= 64        L= 227      L= 3
                                       T= 0.0296    T= 0.00354   T= 0.0463   T= 0.00051 
                                       NA= 214      NA= 37       NA= 82      NA= 4
                                                                             
    A* (linear)                        L= 17        L= 11        L= 20       L= 3
                                       T= 0.294     T= 0.0112    T= 4.0395   T= 0.00103 
                                       NA= 835      NA= 74       NA= 3230    NA= 4
                                                                             
    A* (manhattan)                     L= 17        L= 11        L= 20       L= 3 
                                       T= 0.0569    T= 0.00203   T= 0.366    T= 0.00051
                                       NA= 278      NA= 28       NA= 931     NA= 4
                                       
    H3                                 L= 19        L= 11        L= 20       L= 3
                                       T= 0.0138    T= 0.00665   T= 0.0847   T= 0.000515 
                                       NA= 87       NA= 43       NA= 324     NA= 4

 -----------------------------------------------------------------------------------------

Sin embargo, esta heurística no es admisible. Lo comprobamos en el estado anterior:

          +---+---+---+
          | 1 | 2 | 3 |
          +---+---+---+
          | 4 | 5 | 0 |
          +---+---+---+
          | 7 | 8 | 6 |
          +---+---+---+

La puntuación con 'sequential' es 3. Por otro lado, todas las piezas están en su sitio menos 6, que está a una distancia de 1 de su posición, luego la puntuación con 'manhattan' es 1.

En definitiva, la puntuación con 'h3' es 4 (3 + 1), que es mayor que el coste real de llegar al estado objetivo: este coste es 1, pues basta con intercambiar el 6 con el hueco para resolver el puzzle.

Por tanto, la heurística sobreestima el coste real del estado; luego no es admisible.


### Ejercicio 8. Puzle de 15.
En 1878, Sam Loyd daba un premio de 1.000 dolares a quien fuera capaz de resolver su famoso 15 Puzzle (n=16). Se trataba de un puzzle deslizante de 16 piezas que *no tenía solución* debido que se necesitaba un número impar de movimientos y, como hemos visto, sólo un número de movimientos par tiene solución. Muchas personas trataron de resolverlo. 
El número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. 
En el caso del siguiente estado inicial que sí tiene solución:  (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


Para resolver este ejercicio hemos decidido implementar las clases que hemos utilizdo para el puzzle 3x3 pero adaptado a este nuevo puzle. La clase problema con analizados se puede reutilizar.

In [114]:
class Ocho_Puzzle_Ampliado(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self, estado):
        pos_hueco = estado.index(0) # busco la posicion del 0
        accs=list()
        
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
        
        return accs     

    def result(self, estado, accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        
        ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco - 4]
            l[pos_hueco - 4] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 4]
            l[pos_hueco + 4] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco - 1]
            l[pos_hueco - 1] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco + 1]
            l[pos_hueco + 1] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    
    #redefinimos la funcion coste_de_aplicar_accion para que muestre por pantalla "No se puede realizar esta acción"
    #en caso de producirse una acción que no se puede realizar y 1 si se puede realizar la acción
    def coste_de_aplicar_accion(self, estado, accion):
        if accion in self.actions(estado):
            return 1
        else:
            return "No se puede realizar esta acción."

In [1]:
#Heuristica para el 8 Puzzle ampliado.
def linear_ampliado(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    
    ln = 0
    for i in range(0,16):
            if node.state[i] != 0:
                if node.state[i] != goal[i]: #Si no es el 0 (hueco)
                    ln += 1
                    
    return ln


def manhattan_ampliado(node):
    state = node.state
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    
    mhd = 0
    for i in range(0,16):
        if node.state[i] != 0:
            j = goal.index(state[i])
            if i != j:
                filaEstado = i / 4 
                columnaEstado = i % 4
                filaFinal = j / 4
                columnaFinal = j % 4
                
                mhd += abs(filaEstado - filaFinal) + abs(columnaEstado - columnaFinal)
    
    return mhd

def sqrt_manhattan_ampliado(node):
    ##state = node.state
    
    return math.sqrt(manhattan(node))

def max_heuristic_ampliado(node):
    score1 = manhattan(node)
    score2 = linear(node)
    
    return max(score1, score2)

In [116]:
def resuelve_ocho_puzzle_ampliado(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p = Problema_con_Analizados(Ocho_Puzzle_Ampliado(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol = algoritmo(p8p,h).solution()
        else: 
            sol = algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [117]:
estado_inicial = (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0)

In [118]:
p8 = Ocho_Puzzle_Ampliado(estado_inicial)

In [119]:
p8p = Problema_con_Analizados(p8)

In [122]:
import time
start= time.time()

resuelve_ocho_puzzle_ampliado(estado_inicial,astar_search,manhattan_ampliado)
print(time.time()-start)

Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan_ampliado
Longitud de la solución: 4. Nodos analizados: 7
0.0010106563568115234


Se ha utilizado A* con heurística manhattan

Si lo comparamos con el estado E4 = (1,2,3,0,5,6,4,7,8) evaluado en el ejercicio anterior (que realiza 3 movimientos), Nodos = 4 Longitud = 3 tiempo = 0.00051, observamos que el numero de nodos es relativamente igual, la longitud varía al ser mas grande el numero de estados y su longitud, y el tiempo aumenta en relación a esto.
Para casos sencillos con pocas inversiones, no hay mucha diferencia, pero para estados más complejos aunque no se ha comprobado, es deducible que el puzzle de 16 requerirá más tiempo, la longitud de la solución será mayor y el número de nodos visitados también.